In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate

In [ ]:
funcoes = [
    {
        'deriv_py': lambda x, y: x - y,
        'deriv_latex': 'x - y',
        's_exata': lambda x: x - 1 + 2 * np.exp(-x),
        'init_x': 0,
        'valor_inicial': 1,
        'letra': 'a',
        'intervalo': (0, 1)
    },
    {
        'deriv_py': lambda x, y: -2 * x * y,
        'deriv_latex': '-2xy',
        's_exata': lambda x: np.exp(-x**2),
        'init_x': -2.5,
        'valor_inicial': np.exp(-6.25),
        'letra': 'b',
        'intervalo': (-2.5, 2.5)
    },
    {
        'deriv_py': lambda x, y: 5 * y - 1,
        'deriv_latex': '5y - 1',
        's_exata': lambda x: np.exp(5 * x) + 0.2,
        'init_x': 0,
        'valor_inicial': 1.2,
        'letra': 'c',
        'intervalo': (0, 2)
    },
    {
        'deriv_py': lambda x, y: y - x**2 + 1,
        'deriv_latex': 'y - x^2 + 1',
        's_exata': lambda x: (x + 1)**2 - 0.5 * np.exp(x),
        'init_x': 0,
        'valor_inicial': 0.5,
        'letra': 'd',
        'intervalo': (0, 2)
    },
    {
        'deriv_py': lambda x, y: np.sin(x) / 3 * y,
        'deriv_latex': '\\frac{\\sin(x)}{3y}',
        's_exata': lambda x: np.sqrt((10 / 3) - (2 * np.cos(x) / 3)),
        'init_x': np.pi,
        'valor_inicial': 2,
        'letra': 'e',
        'intervalo': (-1, 1)
    },
]